# 高性能的Python

对于**性能**关键应用，总应该检查两件事：
1. 使用正确的范型
2. 使用正确的高性能库

许多**高性能库**可以加速Python代码的执行：
+ Cython， 用于合并Python和C语言静态编译范型
+ IPython.parallel，用于在本地或者集群上并行执行代码/函数
+ numexpr，用于快速计算
+ multiprocessing，Python内建的（本地）并行处理模块
+ Numba，用于为CPU动态编译Python代码
+ NumbaPro，用于为多核CPU和GPU动态编译Python代码

In [1]:
from timeit import repeat
def perf_comp_data(func_list, data_list, rep=3, number=1):
    '''Function to compare the comperfance of different functions.
    Args:
    func_list: list
        list with function names as strings
    data_list: list
        list with data set names as strings
    rep: int
        number of repetitions of the while comparsion
    number: int
        number of ececutions for every function'''
    res_list = {}
    for name in enumerate(func_list):
        stmt = name[1] + '(' + data_list[name[0]] + ')'  # function to test
        setup = "from __main__ import " + name[1] + ',' + data_list[name[0]]  # setup code
        results = repeat(stmt=stmt, setup=setup, repeat=rep, number=number)  # a list, element is execution time per repetition
        print results
        res_list[name[1]] = sum(results) / rep  # mean
        # print res_list
    res_sort = sorted(res_list.items(), key=lambda p: p[1])  # sorted by time
    print res_sort
    for item in res_sort:
        rel = item[1] / res_sort[0][1]  # relative
        print 'function: ' + item[0] + ', av. time sec: %9.5f, ' % item[1] + 'relative: %6.1f' % rel

    可以把所有的print都注释掉。
    timeit.repeat(stmt='pass', setup='pass', timer=<default timer>, repeat=3, number=1000000)创建一个Timer实例。
    stmt: 需要测量的语句或函数（要执行的代码）
    setup: 初始化代码或构建环境的导入语句（执行代码的准备工作，不计入时间，一般是import之类的）
    timer: 计时函数，平台相关。Windows: time.clock()  Linux: time.time()
    repeat: specifies how many times to call tiemit.timeit()，重复执行次数
    number: specifies the number argument for timeit.timeit(),每一次测量中语句被执行的次数（要执行代码多少遍）
    Returns: 返回一个列表，表示执行每遍的时间（元素个数=repeat）。可以据此取min、max、mean，因为测得的时间是系统经过的时间，有可能被正在运行的其他程序影响，所以min可能最有用。本例中取的是mean。

## 1 Python范型与性能

大数据集上的数值计算相当费时。举个例子，我们想要在包含50万个数值的数组上求取某个复杂数学表达式:

$$y = \sqrt{|cos(x)|}+sin(2+3*x)$$

每次计算都会带来一定的计算负担，除此之外没有任何的特殊含义。


Python函数：

In [2]:
from math import *
def f(x):
    return abs(cos(x)) ** .5 + sin(4+3*x)

使用`range`函数高效地生成一个包含50万个数值的列表对象：

In [3]:
a_list = range(500000)

第一个实现，函数`f1()`在整个数据集上循环，在结果列表对象中附加单独函数求职结果：**（1 包含显式循环的标准Python函数）**

In [4]:
def f1(a):  # loop in the data set
    res = []  
    for x in a:
        res.append(f(x))
    return res

第二种实现，使用迭代：**（2 包含隐含循环的迭代子方法）**

In [5]:
def f2(a):  # iteration
    return [f(x) for x in a]

第三种实现，使用`eval`函数：**（3 包含隐含循环、使用eval的迭代子方法）**

In [6]:
def f3(a):  # function: eval
    ex = 'abs(cos(x)) ** .5 + sin(4+3*x)'
    return [eval(ex) for x in a]

第四种实现，使用Numpy的向量化技术。数据是一个ndarry对象而不是list。没有循环，所以有发生在Numpy级别而不是Python级别：**（4 Numpy向量化技术）**

In [7]:
import numpy as np
a_np = np.array(a_list)
def f4(a):
    return (np.abs(np.cos(a))) ** .5 + np.sin(4+3*a) # element-wise

第五种实现，是使用专门的**numexpr库**来求数值表达式的值，这个库内建了**多线程执行支持**：**（5 numexpr单线程实现）**

In [8]:
import numexpr as ne
def f5(a):
    ex =  'abs(cos(a)) ** .5 + sin(4+3*a)'
    ne.set_num_threads(1)  # single-thread
    return ne.evaluate(ex)

    ne.set_num_threads(nthreads) ->Set a number of threads to be used in operations. Returns the previous setting for the number of threads.
    ne.evaluate(*, **) ->Evaluate a simple array expression element-wise, using the new iterator.

**（6 numexpr多线程实现）**

In [9]:
def f6(a):  # multi-thread
    ex =  'abs(cos(a)) ** .5 + sin(4+3*a)'
    ne.set_num_threads(16)
    return ne.evaluate(ex)

使用`%%time`(给出cell的代码运行一次所花费的时间)计算总执行时间

In [10]:
%%time
r1 = f1(a_list)
r2 = f2(a_list)
r3 = f3(a_list)
r4 = f4(a_np)
r5 = f5(a_np)
r6 = f6(a_np)

Wall time: 9.5 s


In [12]:
np.allclose(r1, r3)

True

In [13]:
np.allclose(r2, r5)

True

    可以用Numpy函数allclose可以轻松地检查两个（类）ndarray对象是否包含相同的数据。

不同实现执行速度的对比：

In [14]:
func_list = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']
data_list = ['a_list', 'a_list', 'a_list', 'a_np', 'a_np', 'a_np']
perf_comp_data(func_list=func_list, data_list=data_list)

[0.385461728395061, 0.36844720987654256, 0.36506074074074135]
[0.36059575308642167, 0.3485317530864229, 0.34361520987654615]
[8.950448592592593, 8.812922469135803, 9.066279901234566]
[0.0565092345679048, 0.05027476543209275, 0.05152908641974818]
[0.015543308641980502, 0.013591308641977662, 0.013274469135801326]
[0.006616098765427125, 0.009174123456794803, 0.00809995061727875]
[('f6', 0.007963390946500226), ('f5', 0.01413636213991983), ('f4', 0.05277102880658191), ('f2', 0.35091423868313026), ('f1', 0.37298989300411495), ('f3', 8.94321698765432)]
function: f6, av. time sec:   0.00796, relative:    1.0
function: f5, av. time sec:   0.01414, relative:    1.8
function: f4, av. time sec:   0.05277, relative:    6.6
function: f2, av. time sec:   0.35091, relative:   44.1
function: f1, av. time sec:   0.37299, relative:   46.8
function: f3, av. time sec:   8.94322, relative: 1123.0


**Summary:**

胜者：多线程`numexpr`实现`f6`。当然，它的速度优势取决于可用核心数量。向量化`numpy`版本`f4`慢于`f5`。纯`Python`实现`f1`和`f2`比胜者慢几十倍。`f3`是最慢的版本，因为对这样大量的求值运算使用`eval`会造成巨大的负担。在`numexpr`的例子中，基于字符串的表达式计算运行一次之后被编译供以后使用；而使用`Python eval`函数，这样一操作要进行50万次。**`numexpr`比`numpy`要快快。在对大数组进行操作时一定要使用`numexpr`**